# Mangrove Coverage: 1988-2024
The code in this file creates an HTML file that visualizes the mangrove forest coverage from 1988 to 2024 on an interactive Earth Engine map. It adds individual layers for each year of the mangrove extent, allowing the user to view and compare areas of mangrove loss. 

In [23]:
# --- Run underlying python scripts
import GEN01_GEE_Authenticate
import GEN02_AOI as  aoi_def
import GEN03_helper_functions as HF
import GEN04_mangrove_layers as ML 
import GEN05_HTML_layout as HTML 
import GEN06_shapefiles as shp


In [24]:
import ee, os

# SERVICE_ACCOUNT = os.environ.get("GEE_SERVICE_ACCOUNT")
# KEY_FILE = "key.json"   # GitHub Actions writes the secret here

# credentials = ee.ServiceAccountCredentials(SERVICE_ACCOUNT, KEY_FILE)
# ee.Initialize(credentials)

# print("✅ Earth Engine initialized with Service Account")

ee.Authenticate()
ee.Initialize(project="gee-mekong-map")

In [25]:
# ------------- Packages & GEE ------------
import ee                                       # Google Earth Engine API for Python
import geemap.foliumap as geemap_folium         # Geemap module integrating Folium maps
import folium                                   # Interactive mapping library based on Leaflet.js
import geemap                                   # Toolkit for working with Google Earth Engine in Python
from geemap import cartoee                      # Module for exporting and visualizing Earth Engine maps using Matplotlib
import ipyleaflet                               # Interactive maps in Jupyter using Leaflet
import ipywidgets as widgets                    # Interactive UI controls for Jupyter notebooks
from branca.element import Element              # Low-level HTML/JS elements for Folium/Branca maps
import geopandas as gpd                         # Spatial data handling (GeoDataFrames)
import glob                                     # File pattern matching (e.g., list all .tif files in a folder)
import json                                     # JSON encoding and decoding (read/write .json files)


In [26]:
# Create a folium-based map
Map = geemap_folium.Map(center=[9.2, 105.75], zoom=11)


# # Add Sentinel-2 as basemap (true color RGB)
# Map.addLayer(
#     ML.sentinel,
#     {"bands": ["B4", "B3", "B2"], "min": 0, "max": 3000},
#     "Sentinel-2 (2024)"
# )

# s2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
#     .filterDate("2024-01-01", "2024-12-31") \
#     .filterBounds(aoi_def.aoi) \
#     .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 10)) \
#     .median()

Map.add_basemap('SATELLITE')


# Map.addLayer(s2, {"bands": ["B4", "B3", "B2"], "min": 300, "max": 2500}, "Sentinel-2 SR (2024)")


In [27]:
# --------------- Dictionary of mangrove coverage masks (updated years) ---------------
mangrove_coverage = {
    "Mangrove coverage 1988": ML.mangrove_1988,
    "Mangrove coverage 1992": ML.mangrove_1992,
    "Mangrove coverage 1997": ML.mangrove_1997,
    "Mangrove coverage 2001": ML.mangrove_2001,
    "Mangrove coverage 2005": ML.mangrove_2005,
    "Mangrove coverage 2010": ML.mangrove_2010,
    "Mangrove coverage 2015": ML.mangrove_2015,
    "Mangrove coverage 2020": ML.mangrove_2020,
    "Mangrove coverage 2024": ML.mangrove_2024,
}

# --------------- Add mangrove coverage layers and generate tile URLs -----------------
mangrove_coverage_layers = {}

for label, img in mangrove_coverage.items():
    # Add to interactive map
    Map.addLayer(
        img.updateMask(img),
        {"palette": [ML.color_mangrove_coverage]},
        label
    )

#     # Generate Earth Engine tile URL
#     tile_url = img.getMapId({"palette": [color_mangrove]})["tile_fetcher"].url_format
#     mangrove_coverage_layers[label] = tile_url

# # (Optional) Export all tile URLs to a JSON file
# import json
# with open("mangrove_coverage_layers.json", "w") as f:
#     json.dump(mangrove_coverage_layers, f, indent=4)


In [28]:
# ------- Add legend      --------
legend_title = "Mangrove Coverage Map (1988-2024)"

HTML.add_folium_legend(Map, legend_title, HTML.legend_dict_mangrove_COVERAGE, style=HTML.style)

# ------- Add scalebar     -------
Map.add_child(HTML.ScaleBar(font_size="14px"))

# ------- Add north arrow --------
HTML.add_north_arrow(Map, position="bottomleft", arrow_size="35px", text_size="25px")

# ------- Add Living LAb logo -----
Map.get_root().html.add_child(folium.Element(HTML.logo_html))

In [29]:
# Add the commune boundaries to the map
folium.GeoJson(
    shp.geojson_commune,
    name="Commune boundaries",
    style_function=shp.commune_style,
    ).add_to(Map)


#  Add the sea dikes group to the map
shp.sea_dikes_group.add_to(Map)


# Add the breakwater group to the map
shp.breakwaters_group.add_to(Map)

# other human activities
shp.Extra_group.add_to(Map)
folium.LayerControl().add_to(Map)   

In [ ]:
Map.to_html("Mangrove_COVERAGE_map.html")
